Esse notebook foi feito para gerar mel-espectrogramas usando a biblioteca NVIDIA’s Data Loading Library.

O código abaixo instala a biblioteca:

In [ ]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist nvidia-dali-cuda100

In [ ]:
#Importando bibliotecas
import os
import librosa as librosa
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import librosa.display
import pandas as pd
import cv2
import timeit

import warnings
warnings.filterwarnings('ignore')

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
import nvidia.dali as dali

A função abaixo recebe um mel-espectrograma e retorna uma array que representa uma imagem. Será usado mais abaixo.

In [ ]:
def mono_to_color(X: np.ndarray,
                  mean=None,
                  std=None,
                  norm_max=None,
                  norm_min=None,
                  eps=1e-6):
    """
    Code from https://www.kaggle.com/daisukelab/creating-fat2019-preprocessed-data
    """
    # Stack X as [X,X,X]
    #X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

O código em baixo é uma maneira mais rápida de carregar os audios com o librosa

In [ ]:
def get_clip_sr(path,offset=0,duration=None):
    clip, sr_native = librosa.core.audio.__audioread_load(path, offset=offset, duration=duration, dtype=np.float32)
    clip = librosa.to_mono(clip)
    sr = 22050
    if sr_native > 0:
        clip = librosa.resample(clip, sr_native, sr, res_type='kaiser_fast')
    return clip, sr

Aqui é definido o pipeline para a geração de espectrogramas:

In [ ]:
class MelSpectrogramPipeline(Pipeline):
    def __init__(self,full_path, device, batch_size, nfft, window_length, window_step, num_threads=1, device_id=0):
        super(MelSpectrogramPipeline, self).__init__(batch_size, num_threads, device_id)
        self.device = device

        self.batch_data = []
        y,sr =  get_clip_sr(full_path,0,5);
        for _ in range(batch_size):
            self.batch_data.append(np.array(y, dtype=np.float32))

        self.external_source = ops.ExternalSource()
        self.spectrogram = ops.Spectrogram(device=self.device,
                                           nfft=nfft,
                                           window_length=window_length,
                                           window_step=window_step)

        self.mel_fbank = ops.MelFilterBank(device=self.device,
                                           sample_rate=sr,
                                           nfilter = 310,
                                           freq_high = 10300,)

        self.dB = ops.ToDecibels(device=self.device,
                                 multiplier = 10.0,
                                 cutoff_db = -80)

    def define_graph(self):
        self.data = self.external_source()
        out = self.data.gpu() if self.device == 'gpu' else self.data
        out = self.spectrogram(out)
        out = self.mel_fbank(out)
        out = self.dB(out)
        return out

    def iter_setup(self):
        self.feed_input(self.data, self.batch_data)

Abaixo estou iterando por todos as linhas presentes no arquivo train.csv e extraindo os mel-espectrogramas e gerando imagens para os 5s iniciais de cada audios pertencente a primeira espécie.

In [ ]:
PATH_TRAIN = "../input/segmentation-on-cornell-dataset-exemple"
species = os.listdir(PATH_TRAIN)
species.sort()
species = species[3:]

In [ ]:
#Declarando variavel para contar o tempo
start = timeit.default_timer()

for sp in species:
    print(sp)
    if(os.path.isdir(sp)):
        continue
    
    if(not(os.path.isdir(PATH_TRAIN + '/' + sp))):
       continue
    
    os.mkdir(sp)
    
    for chunk in os.listdir(PATH_TRAIN + '/' + sp):    
        #Montando o caminho do audio    
        full_path = PATH_TRAIN + '/' + sp +'/' + chunk
    
        pipe = MelSpectrogramPipeline(full_path,device='cpu', batch_size=1, nfft=1536, window_length=1536, window_step=360)
        pipe.build()
        outputs = pipe.run()
        S_dB = outputs[0].at(0)
    
        #Ajustando imagem
        im = mono_to_color(S_dB);
        im = cv2.resize(im, (224,224))
        im = cv2.flip(im, 0)
    
        #Escrevendo imagem
        cv2.imwrite('{}/{}.png'.format(sp,str(chunk[:-4])),im)

#contador final do tempo    
stop = timeit.default_timer()

print('Time: ', stop - start) 

Exemplo de visualização das imagens geradas.

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('./yetvir/38365.png')
imgplot = plt.imshow(img)